In [ ]:
%matplotlib widget

In [ ]:
from ibvpy.api import TStepBC
from ibvpy.bcond import BCDof, BCSlice
from ibvpy.api import XDomainFEGrid
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import MATS2DElastic
from ibvpy.tmodel.viz3d_tensor_field import \
   Vis3DTensorField, Viz3DTensorField

In [ ]:
B = 1
fe_grid = XDomainFEGrid(integ_factor = B,
                        coord_min=(0,0),
                        coord_max=(1,1),
                        shape=(10,10),
                        fets=FETS2D4Q())

In [ ]:
mat_elastic = MATS2DElastic(E=28000, nu=0.2)

In [ ]:
ts = TStepBC(
    domains = [(fe_grid, mat_elastic)]
)

In [ ]:
fix_left = BCSlice(slice=fe_grid.mesh[0, 3:, 0, :], var='u',
                   dims=[0,1], value=0 )
load_right = BCSlice(slice=fe_grid.mesh[-1,:,-1,:],
                    var='u', dims=[1], value=0.1)
ts.bc = [fix_left, load_right]

In [ ]:
ts.hist.vis_record = {
    'stress' : Vis3DTensorField(var='sig_ab')
}

In [ ]:
sim = ts.sim
sim.tloop.verbose=True
sim.tline.step=1
sim.reset()
sim.run()

In [ ]:
U_1 = ts.hist.U_t[-1]

In [ ]:
eps_Emab = fe_grid.map_U_to_field(U_1)

In [ ]:
eps_Emab[3,0]

In [ ]:
sig_tip, _ = mat_elastic.get_corr_pred(eps_Emab[3,:],1)

In [ ]:
sig_tip

In [ ]:
if True:
    from mayavi import mlab
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False
    f_stress = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    stress_viz = Viz3DTensorField(vis3d=ts.hist['stress'])
    stress_viz.setup()
    stress_viz.plot(ts.t_n)
    mlab.show()